# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do


In [1]:
import dlt
import duckdb

# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.


In [2]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

In [3]:
# Example usage:
limit = 5
generator = square_root_generator(limit)

output_sum = 0

for sqrt_value in generator:
    output_sum += sqrt_value

print("Sum of the outputs of the generator for limit = 5: ", output_sum)

Sum of the outputs of the generator for limit = 5:  8.382332347441762


In [4]:
# Example usage:
limit = 15
generator = square_root_generator(limit)

output = [sqrt_value for sqrt_value in square_root_generator(limit)]


print("13th number yielded by the generator: ", output[12])

13th number yielded by the generator:  3.605551275463989


# 2. Append a generator to a table with existing data

Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**


In [5]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}


for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)

{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [6]:
people_1_data = [person for person in people_1()]
people_2_data = [person for person in people_2()]

In [7]:
pipeline = dlt.pipeline(
    dataset_name='peoples_data',
    pipeline_name='peoples_pipeline',
    destination='duckdb'
)

# load data from people_1_data
load_data_info = pipeline.run(
    data=people_1_data,
    table_name='peoples'
)

print(load_data_info)

Pipeline peoples_pipeline load step completed in 0.50 seconds
1 load package(s) were loaded to destination duckdb and into dataset peoples_data
The duckdb destination used duckdb:////workspaces/zoomcamp-data-engineer-2024/workshop_dlt/peoples_pipeline.duckdb location to store data
Load package 1707892130.9910827 is LOADED and contains no failed jobs


In [8]:
# Append data from people_2_data
append_data_info = pipeline.run(
    data=people_2_data,
    table_name='peoples',
    write_disposition='append'
)

print(append_data_info)

Pipeline peoples_pipeline load step completed in 0.39 seconds
1 load package(s) were loaded to destination duckdb and into dataset peoples_data
The duckdb destination used duckdb:////workspaces/zoomcamp-data-engineer-2024/workshop_dlt/peoples_pipeline.duckdb location to store data
Load package 1707892138.56721 is LOADED and contains no failed jobs


In [13]:
conn = duckdb.connect(f'{pipeline.pipeline_name}.duckdb')
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
# display(conn.sql("DESCRIBE"))
data_table = conn.sql("SELECT * FROM peoples").df()
data_table.head()
sum_of_ages = conn.sql('SELECT SUM(age) total_ages FROM peoples').df()
sum_of_ages.head()

,total_ages
0,353.0


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


In [14]:
pipeline_merge = dlt.pipeline(
    dataset_name='people_merged_data',
    pipeline_name='people_merge_pipeline',
    destination='duckdb'
)

load_info_1 = pipeline_merge.run(
    data=people_1_data,
    table_name='peoples'
)

print(load_info_1, '\n')

load_info_2 = pipeline_merge.run(
    data=people_2_data,
    table_name='peoples',
    write_disposition='merge',
    primary_key='ID'
)

print(load_info_2)

Pipeline people_merge_pipeline load step completed in 0.67 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_merged_data
The duckdb destination used duckdb:////workspaces/zoomcamp-data-engineer-2024/workshop_dlt/people_merge_pipeline.duckdb location to store data
Load package 1707892969.428975 is LOADED and contains no failed jobs 

Pipeline people_merge_pipeline load step completed in 0.86 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_merged_data
The duckdb destination used duckdb:////workspaces/zoomcamp-data-engineer-2024/workshop_dlt/people_merge_pipeline.duckdb location to store data
Load package 1707892970.784673 is LOADED and contains no failed jobs


In [18]:
conn = duckdb.connect(f'{pipeline_merge.pipeline_name}.duckdb')
conn.sql(f"SET search_path = '{pipeline_merge.dataset_name}'")

# all_data = conn.sql('select * from peoples').df()
# all_data.head(10)

sum_of_age = conn.sql('SELECT SUM(age) total_ages FROM peoples').df()
sum_of_age.head()

,total_ages
0,266.0
